<a href="https://colab.research.google.com/github/Gregory-Eales/LANL-Earthquake-Prediction/blob/master/LANL_Earthquake_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LANL Earthquake Prediction on Colab

## To Do:
- Create Data  Loading Functions
- Create Grid Search Functions: Dimensions (model, parameters,  data input size)
- 

## Setting up

### Import Dependencies

In [1]:
# import dependencies
import numpy as np
import pandas as pd
import tensorflow as tf

C:\Users\Greg\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
!ls

LANL-Earthquake-PredictionÂ¶.ipynb
LANL_Earthquake_Colab.ipynb
LANL_Earthquake_New.ipynb
README.md
TrainingDataSegmentation.py
Weights_DataSize_10000_LSTM_Model_01.h5
Weights_DataSize_10000_LSTM_Model_02.h5
Weights_DataSize_10000_LSTM_Model_03.h5
Weights_DataSize_10000_LSTM_Model_04.h5


### Define Data Loading Functions

In [3]:
# set data paths from drive
training_data_path = "/Users/Greg/Desktop/LANL-Data/LANL-Training-Data/"
testing_data_path = "/Users/Greg/Desktop/LANL-Data/LANL-Test-Data/"


def get_file_number(n):
    n = str(n)
    n = "0"*(3-len(n)) + n
    return n

def load_data(number):
    path = "/Users/Greg/Desktop/LANL-Data/LANL-Training-Data/Training-Data"
    path = path + get_file_number(number) + ".csv"
    train_df = pd.read_csv(path, dtype={'acoustic_data': np.int16, 'time_to_failure': np.float32})
    print(train_df.head())
    return train_df

def create_training_data(number, data_length):
    train_df = load_data(number)
    samples = train_df.shape[0]//data_length
    X = train_df["acoustic_data"].as_matrix()
    Y = train_df["time_to_failure"].as_matrix()
    del train_df
    train_df = None
    x_train = []
    y_train = []
    for i in range(samples):
        x_train.append(X[i*data_length:(i+1)*data_length].tolist())
        y_train.append(np.sum(Y[i*data_length:(i+1)*data_length].tolist()))
    x_train.append(X[-1*data_length -1:-1].tolist())
    y_train.append(np.sum(Y[-1*data_length-1:-1]))
    print(type(x_train), len(x_train[-1]))
    x_train = np.reshape(np.array(x_train), [samples+1, data_length, 1])
    y_train = np.reshape(np.array(y_train), [samples+1, 1])
    y_train = y_train/data_length
    return x_train, y_train

In [4]:
x, y = create_training_data(1, 10000)

   acoustic_data  time_to_failure
0              4          11.5408
1              5          11.5408
2              6          11.5408
3              3          11.5408
4              4          11.5408


C:\Users\Greg\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\Greg\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


<class 'list'> 10000


## Model Evaluation

### LSTM Neural Networks

In [5]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Masking, Embedding, RNN

Using TensorFlow backend.


In [6]:
# define LSTM Models for training

# LSTM 01
LSTM_Model_01 = Sequential()
LSTM_Model_01.add(LSTM(15, input_dim=1, return_sequences=True))
LSTM_Model_01.add(LSTM(15))
LSTM_Model_01.add(Dense(15, activation="relu"))
LSTM_Model_01.add(Dense(5, activation="relu"))
LSTM_Model_01.add(Dense(1, activation="relu"))
LSTM_Model_01.compile(optimizer="adam", loss='mean_squared_error')

# LSTM 02
LSTM_Model_02 = Sequential()
LSTM_Model_02.add(LSTM(15, input_dim=1, return_sequences=True))
LSTM_Model_02.add(LSTM(15))
LSTM_Model_02.add(Dense(15, activation="relu"))
LSTM_Model_02.add(Dense(1, activation="relu"))
LSTM_Model_02.compile(optimizer="adam", loss='mean_squared_error')

# LSTM 03
LSTM_Model_03 = Sequential()
LSTM_Model_03.add(LSTM(15, input_dim=1, return_sequences=True))
LSTM_Model_03.add(LSTM(15, return_sequences=True))
LSTM_Model_03.add(LSTM(15))
LSTM_Model_03.add(Dense(1, activation="relu"))
LSTM_Model_03.compile(optimizer="adam", loss='mean_squared_error')

# LSTM 04
LSTM_Model_04 = Sequential()
LSTM_Model_04.add(LSTM(15, input_dim=1, return_sequences=True))
LSTM_Model_04.add(LSTM(15, return_sequences=True))
LSTM_Model_04.add(LSTM(15, return_sequences=True))
LSTM_Model_04.add(LSTM(15))
LSTM_Model_04.add(Dense(1, activation="relu"))
LSTM_Model_04.compile(optimizer="adam", loss='mean_squared_error')


LSTM_Models = [LSTM_Model_01, LSTM_Model_02, LSTM_Model_03, LSTM_Model_04]

C:\Users\Greg\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  """
C:\Users\Greg\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(15, return_sequences=True, input_shape=(None, 1))`
  """
C:\Users\Greg\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  
C:\Users\Greg\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(15, return_sequences=True, input_shape=(None, 1))`
  
C:\Users\Greg\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
C:\Users\Greg\Anaconda3\lib\site-packages\ipykernel_l

In [46]:

def train_LSTM_Models(LSTM_Models):
  
  for file_num in range(14):
    data_size = 10000
    x_train, y_train = create_training_data(file_num, data_size)
    for number, lstm_model in enumerate(LSTM_Models):
      lstm_model.load_weights("Weights_DataSize_" + str(data_size) + "_LSTM_Model_0" + str(number+1) + ".h5")
      lstm_model.fit(x_train, y_train)
      lstm_model.save_weights("Weights_DataSize_" + str(data_size) + "_LSTM_Model_0" + str(number+1) + ".h5")
      

In [47]:
train_LSTM_Models(LSTM_Models)

   acoustic_data  time_to_failure
0             12           1.4691
1              6           1.4691
2              8           1.4691
3              5           1.4691
4              8           1.4691


C:\Users\Greg\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\Greg\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


<class 'list'> 10000
Epoch 1/1
566/566 [==============================] - 308s 544ms/step - loss: 0.7197
Epoch 1/1
566/566 [==============================] - 309s 547ms/step - loss: 0.7113
Epoch 1/1
566/566 [==============================] - 466s 823ms/step - loss: 0.6010
Epoch 1/1
566/566 [==============================] - 633s 1s/step - loss: 0.4315
   acoustic_data  time_to_failure
0              4          11.5408
1              5          11.5408
2              6          11.5408
3              3          11.5408
4              4          11.5408
<class 'list'> 10000
Epoch 1/1
4443/4443 [==============================] - 2367s 533ms/step - loss: 44.4004
Epoch 1/1
4443/4443 [==============================] - 2384s 536ms/step - loss: 17.7467
Epoch 1/1
4443/4443 [==============================] - 3608s 812ms/step - loss: 16.6095
Epoch 1/1
4443/4443 [==============================] - 4862s 1s/step - loss: 14.8333
   acoustic_data  time_to_failure
0              1          14.1806
1   

KeyboardInterrupt: 

In [8]:
LSTM_Models[1].load_weights("Weights_DataSize_" + str(10000) + "_LSTM_Model_0" + str(2) + ".h5")
LSTM_Models[1].predict(x[0:1])

array([[4.5362196]], dtype=float32)

In [9]:
y[0:1]

array([[11.53994964]])

In [ ]:
def Make_LSTM_Submission(LSTM_Models):
    
    for i, lstm_model in enumerate(LSTM_Models):
        

### Feed Forward Neural Network

### Support Vector Machines

### Neural Ordinary Differential Equations

## Training Final Model